# Example Portfolio Factor Analysis
This notebook walks through the modular steps of **PortfolioFactorAnalysis**, illustrating each analysis with `plotly` visualizations.


## 1. Imports and Setup
We import required libraries, load the `PortfolioFactorAnalysis` class, and set up our date range and (optional) portfolio weights.

In [1]:

#comment out for local testing out of Main Sequence Platform
import dotenv
import numpy as np
dotenv.load_dotenv('../../.env')


import datetime
import pandas as pd
import plotly.express as px
import mainsequence.client as ms_client
import os
os.environ["LOG_LEVEL_STDOUT"]="INFO" #override info level for notebook
# Import the analysis class and time-series helpers
from factorinvesting.data_nodes.factors_time_series import (FactorReturnsDataNodes,
                                                            )
from factorinvesting.src.analysis import PortfolioFactorAnalysis

market_asset = ms_client.Asset.get(ticker="IVV",
                                   security_type=ms_client.MARKETS_CONSTANTS.FIGI_SECURITY_TYPE_ETP,
                                   security_market_sector=ms_client.MARKETS_CONSTANTS.FIGI_MARKET_SECTOR_EQUITY,
                                   )
factor_returns_ts = FactorReturnsDataNodes(assets_category_unique_id='s&p500_constitutents',
                                                market_beta_asset_proxy=market_asset,
                                                )
# Specify analysis date range
start = datetime.datetime(2022, 1, 1)
end = datetime.datetime(2025, 1, 30)


portfolio_weights = PortfolioFactorAnalysis.create_random_portfolio()
# Instantiate and run all analyses
pfa = PortfolioFactorAnalysis(
    factor_returns_ts=factor_returns_ts,
    portfolio_weights=portfolio_weights,
    start_date=start,
    end_date=end
)

python-dotenv could not parse statement starting at line 3
python-dotenv could not parse statement starting at line 4
python-dotenv could not parse statement starting at line 5
python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8
2025-08-11T10:11:41.594052Z [info     ] Set remote data source to orm_class='DataSource' id=2 display_name='Default DB' organization=2 class_type='timescale_db' status='AVAILABLE' extra_arguments=None application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at models_tdag.py:1996 in set_remote_db())


Get pod configuration
Executing Script: 
# -- Auto-generated imports for time_series --
# -- Auto-generated imports for rebalance_strategies --
# -- Auto-generated imports for apps --

from mainsequence.virtualfundbuilder.agent_interface import TDAGAgent
print('Initialize TDAGAgent')
tdag_agent = TDAGAgent()
Initialize TDAGAgent


2025-08-11T10:11:50.678288Z [info     ] Setup TDAG Agent successfull   application_name=ms-sdk command_id=None data_source_id=2 job_run_id=None project_id=1 (at agent_interface.py:28 in __init__())


Inspecting polygon package contents:
['AuthError', 'BadResponse', 'RESTClient', 'WebSocketClient', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', 'exceptions', 'logging', 'modelclass', 'rest', 'version', 'websocket']


## 2. Load Exposures
Load per-asset factor exposures into `exposures_df`.

$$ X_{i,f}(t) = \text{exposure of asset }i\text{ to factor }f \text{ at time }t$$

In [2]:
pfa.portfolio_weights

BBG000N9MNX3    0.074286
BBG000B9XRY4    0.098472
BBG000BVPV84    0.161695
BBG009S39JX6    0.218856
BBG000BBJQV0    0.194392
BBG000BPH459    0.208248
BBG000MM2P62    0.044051
Name: weight, dtype: float64

## 3. Compute Portfolio Exposures Over Time
Aggregate asset exposures into portfolio exposures:

$$ x_f(t) = \sum_i w_i \, X_{i,f}(t) $$

In [3]:
# 3. Compute portfolio exposures
port_expo = pfa.portfolio_exposure_df.reset_index().melt(
    id_vars='time_index', var_name='Factor', value_name='Exposure'
)
fig = px.line(port_expo, x='time_index', y='Exposure', color='Factor',
              title='Portfolio Factor Exposures Over Time')
fig.show()

## 4. Rolling Statistics
Calculate rolling mean, volatility, and autocorrelation of portfolio exposures:

$$ \mu_{f,t} = \frac{1}{W}\sum_{k=0}^{W-1} x_f(t-k),\quad \sigma_{f,t} = \sqrt{\frac{1}{W-1}\sum_{k=0}^{W-1}(x_f(t-k) - \mu_{f,t})^2},\quad \rho_{f,t} = \mathrm{corr}(x_f(t), x_f(t-1)) $$

In [4]:
# 4. Rolling statistics
rm, rv, racf = pfa.rolling_statistics(window=60)
# Plot rolling mean for first three factors
rm_df = rm.reset_index().melt(id_vars='time_index', var_name='Factor', value_name='RollingMean')
fig = px.line(rm_df[rm_df.Factor.isin(rm.columns[:3])], x='time_index', y='RollingMean', color='Factor',
              title='60-Day Rolling Mean of Factor Exposures')
fig.show()

## 5. Factor Attribution
Compute per-factor P&L contributions and predicted portfolio return:

$$ C_f(t) = x_f(t)\,r_f(t),\quad \hat{R}(t) = \sum_f C_f(t) $$

In [5]:
# 5. Load factor returns and perform attribution
contrib, pred_returns = pfa.factor_attribution()
last_date = contrib.index.max()
# Plot contributions for last date
bar = contrib.loc[last_date].reset_index(name='Contribution')
fig = px.bar(bar, x='index', y='Contribution',
             title=f'Factor P&L Contributions on {last_date.date()}')
fig.show()

## 6. Cross-Sectional Tail Metrics

Because each factor exposure is **z-scored cross-sectionally every day**, the
standard deviation of \(X_{i,f}(t)\) is forced to be ≈ 1 for *every* factor on
*every* date, so a “dispersion” plot is uninformative.

Instead we examine the **tails** of the exposure distribution:

* **Positive-tail share**
  \[
    \text{tail\_pos}_f(t)=\frac{1}{N}\sum_{i=1}^{N}
      \mathbf 1\!\bigl[X_{i,f}(t) > \tau\bigr]
  \]

* **Negative-tail share**
  \[
    \text{tail\_neg}_f(t)=\frac{1}{N}\sum_{i=1}^{N}
      \mathbf 1\!\bigl[X_{i,f}(t) < -\tau\bigr]
  \]

with threshold \(\tau=2\) ( assets whose exposure is more than
two standard-deviations from the mean).
A high tail share signals **crowding** or **polarisation** in that factor on
date \(t\).


In [6]:
# 6. Cross-sectional tail metrics
tail_metrics = pfa.exposure_tail_metrics(last_date, tail_cut=2.0)

# Bar chart: % of assets in + and – tails
import plotly.express as px
plot_df = tail_metrics.reset_index().melt(id_vars='index',
                                          value_vars=['tail_pos', 'tail_neg'],
                                          var_name='Tail', value_name='Percent')
fig = px.bar(plot_df,
             x='index', y='Percent', color='Tail', barmode='group',
             title=f'Fraction of Assets with |Exposure| > 2 on {last_date.date()}',
             labels={'index':'Factor'})
fig.update_layout(yaxis_tickformat='.0%', yaxis_title='% of Universe')
fig.add_hline(y=0, line_color='black')
fig.show()


## 7. Exposure Correlation Matrix
Examine correlations across assets based on their factor exposures.

In [7]:
# 7. Correlation matrix
corr = pfa.correlation_matrix(last_date)
fig = px.imshow(corr, labels=dict(x="Asset", y="Asset"), title=f'Exposure Correlation Matrix on {last_date.date()}')
fig.show()

In [8]:
pd.Series(np.random.randn(len(contrib.columns)), index=contrib.columns)


lncap            -1.498978
lncap2            1.206049
beta             -1.217627
nl_beta           0.142504
mom_12_1         -0.254858
mom_1m            0.171139
resid_vol        -0.625156
liquidity        -1.742637
book_to_price    -0.675716
leverage          0.787063
div_yield         0.740564
earnings_yield   -0.310009
profitability     0.551119
dtype: float64

## 8. Scenario Analysis
Simulate portfolio P&L under a hypothetical factor shock:

$$ \Delta \text{P\&L}_f = x_f(t)\,\Delta r_f^{\mathrm{shock}} $$

In [9]:
# 8. Scenario analysis
shock = pd.Series(np.random.randn(len(contrib.columns)), index=contrib.columns)
impact = pfa.scenario_analysis(shock, last_date)
impact_df = impact.reset_index(name='PnlImpact')
fig = px.bar(impact_df, x='index', y='PnlImpact', title='Hypothetical Shock P&L Impact')
fig.show()

## 9. Estimating Realized Factor Premia via Cross‐Sectional Regression

In practice we want to recover the *realized* factor returns (risk premia) $\beta_f(t)$ on each date $t$ by exploiting the cross‐section of asset returns and exposures.  This is the classic Fama–MacBeth (1973) “second‐pass” regression applied daily.

---

### Model Specification

We assume that on date $t$, each asset’s excess return $r_i(t)$ is driven by its exposures $X_{i,f}(t)$ to $K$ risk factors plus idiosyncratic noise:

$$
r_i(t) \;=\; \alpha(t)\;+\;\sum_{f=1}^K X_{i,f}(t)\,\beta_f(t)\;+\;\varepsilon_i(t),
$$

where:

- $\alpha(t)$ is the cross‐sectional intercept capturing any common shift (“alpha”) not explained by the factors.
- $\beta_f(t)$ is the *realized* daily risk premium for factor $f$ at date $t$.
- $\varepsilon_i(t)$ is the idiosyncratic residual for asset $i$ on date $t$.

---

### Estimation Procedure

1. **Snapshot Data**
   - Build matrix $X(t)\in\mathbb{R}^{N\times K}$ with row $i$ = $[X_{i,1}(t),\dots,X_{i,K}(t)]$.
   - Stack returns into vector $\mathbf{r}(t)\in\mathbb{R}^N$.

2. **Add Intercept**
   - Prepend a column of ones to $X(t)$ to estimate $\alpha(t)$ simultaneously with $\beta(t)$.

3. **Weighted Least Squares** (optional)
   - If you have asset‐level weights $w_i$ (e.g.\ market cap), solve
     $$
     \min_{\alpha,\beta}\;
     \bigl[\mathbf{r}(t) - \alpha\,\mathbf{1} - X(t)\,\beta\bigr]^\top
     \,\mathrm{diag}(w)\,
     \bigl[\mathbf{r}(t) - \alpha\,\mathbf{1} - X(t)\,\beta\bigr].
     $$
   - Otherwise use ordinary OLS.

4. **Robust Inference**
   - Compute heteroskedasticity‐consistent (HC1) standard errors for valid $t$‐tests even if residuals exhibit non‐constant variance.

5. **Diagnostic Checks**
   - **Condition Number** of $X^\top X$ to flag multicollinearity among factors.
   - **R-squared** to gauge how well the factors explain the cross‐section of returns.

---

### Economic Interpretation

- **$\beta_f(t)$**: tells you *ex‐post* how much you would have earned per unit of exposure to factor $f$ on that day.
- **$\alpha(t)$**: measures any systematic return not captured by your factor set—often interpreted as model misspecification or omitted factors.
- **Residuals $\varepsilon_i(t)$**: idiosyncratic shocks; aggregating them weighted by your portfolio weights gives your specific‐risk P&L.

---

### References

- Fama, E. F. & MacBeth, J. D. (1973). “Risk, Return, and Equilibrium: Empirical Tests,” *Journal of Political Economy*, 81(3), 607–636.
- Cochrane, J. H. (2005). *Asset Pricing* (Chapter 6: “The Cross‐Section of Expected Returns”), Princeton University Press.
- Axioma/Barra white paper (2006). “Axioma Risk Models: Construction and Implementation,” Axioma, Inc.


In [10]:
# 9. Estimate realized factor premia and visualize



# 2) Estimate factor premia with robust errors
res = pfa.estimate_realized_factor_premia(last_date, asset_weights=None)

# 3) Prepare DataFrame for plotting
premia = res['premia']
stderr = res['stderr']
plot_df = pd.DataFrame({
    'Factor': premia.index,
    'EstimatedReturn': premia.values,
    'StdErr': stderr.values
})

# 4) Plot with error bars
import plotly.express as px

fig = px.bar(
    plot_df,
    x='Factor',
    y='EstimatedReturn',
    error_y='StdErr',
    title=f'Realized Factor Premia on {last_date.date()}',
    labels={'EstimatedReturn': 'Estimated Premium', 'Factor': 'Factor'}
)
fig.add_hline(y=0, line_dash="dash", line_color="black")
fig.update_layout(yaxis_title='Premium', xaxis_title='Factor')
fig.show()


## Computing the Factor Covariance Matrix

In order to forecast portfolio risk and allocate risk budgets, we need an ex‐ante estimate of the factor covariance matrix $\Sigma_f(t)$. Below are three common estimators used in Axioma/Barra-style risk models:

---

### 1. Rolling-Window Sample Covariance
Let $\mathbf{r}_t = [r_{1,t}, \dots, r_{K,t}]^\top$ be the vector of $K$ factor returns at time $t$. Over a lookback window of $W$ days, the rolling-window sample covariance is

$$
\widehat{\Sigma}_{\text{sample}}(t)
= \frac{1}{W - 1}
\sum_{k = t - W + 1}^{t}
\bigl(\mathbf{r}_k - \bar{\mathbf r}\bigr)\,
\bigl(\mathbf{r}_k - \bar{\mathbf r}\bigr)^\top,
$$

where $\bar{\mathbf r}$ is the time-series mean of each column of returns.

---

### 2. Exponentially-Weighted Covariance (EWMA)
Axioma typically applies an EWMA estimator with half-life $h$ (often 63 days). Define weights

$$
\omega_k = \exp\!\bigl(-\ln(2)\,\frac{t - k}{h}\bigr),
\qquad k = 1,\dots,t,
$$

and then

$$
\widehat{\Sigma}_{\mathrm{EWMA}}(t)
= \frac{\sum_{k=1}^t \omega_k\,
       \bigl(\mathbf{r}_k - \bar{\mathbf r}\bigr)
       \bigl(\mathbf{r}_k - \bar{\mathbf r}\bigr)^\top}
       {\sum_{k=1}^t \omega_k}.
$$

---

### 3. Shrinkage Toward a Constant-Correlation Target
Barra stabilizes the EWMA estimate by shrinking toward a constant-correlation target. Let:

- $\Sigma_0 = \widehat{\Sigma}_{\mathrm{EWMA}}(t)$
- $D = \mathrm{diag}(\Sigma_0)$
- $\bar\rho =$ average off-diagonal correlation in $\Sigma_0$

The constant-correlation target is

$$
T
= \sqrt{D}\,\bigl(\bar\rho\,\mathbf{1}\mathbf{1}^\top + (1-\bar\rho)\,I\bigr)\,\sqrt{D},
$$

and the shrunk covariance is

$$
\Sigma_{\mathrm{shrunk}}(t)
= \lambda\,\Sigma_0 \;+\;(1 - \lambda)\,T,
$$

where $\lambda\in[0,1]$ controls the shrinkage intensity.

---

#### Putting It All Together
In practice, generate a full time-series of $\Sigma_f(t)$ for each rebalancing date using your chosen method (`sample`, `ewma`, or `ewma+shrinkage`), then feed $\Sigma_f(t)$ into your ex-ante risk and risk-contribution calculations.




Barra’s **constant-correlation** shrinkage is a pragmatic, domain-driven choice rather than a purely statistical optimum like Ledoit–Wolf. The reasons they (and Axioma) do it this way are:

---

### Business-driven target

A constant-correlation matrix

$$
T_{ij} = \bar\rho\,\sigma_i\,\sigma_j
$$

reflects the idea that, absent strong evidence to the contrary, all factors tend to move together to some baseline degree $\bar\rho$.
This ties back to economic beliefs (e.g. in a systemic “beta” regime, everything co-moves) rather than just the data.

---

### Stability of off-diagonals

Raw sample or EWMA covariances (especially with limited history) suffer noisy off-diagonal estimates.
Shrinking toward a simple constant-correlation target aggressively tames that noise, producing more stable risk estimates.

---

### Domain simplicity and interpretability

The single parameter $\bar\rho$ is easy to explain and calibrate (e.g. to a long-term average).
Ledoit–Wolf’s optimum shrinkage intensity and target are data-driven and can fluctuate wildly if market regimes shift, whereas Barra’s approach remains consistent unless you deliberately re-estimate $\bar\rho$.

---

### Historical lineage and implementation

Barra’s risk model predates modern random-matrix theory-based estimators. Their teams built in a constant-correlation or single-index target because it aligned with how their clients understood and managed factor risk.
It was efficient to compute and easy to embed in production, long before Ledoit–Wolf routines were common in statistical packages.

In [31]:

# Compute the three variants of Σ_f(t) for the most recent date
cov_sample  = pfa.compute_factor_covariance(method='sample', window=252)
cov_ewma    = pfa.compute_factor_covariance(method='ewma',   halflife=63)
cov_shrunk  = pfa.compute_factor_covariance(method='ewma',   halflife=63, shrinkage=0.8)

# Visualize each as a heatmap
import plotly.express as px

fig1 = px.imshow(
    cov_sample,
    labels={'x':'Factor i','y':'Factor j','color':'Covariance'},
    title='Rolling‐Window Sample Covariance (Latest)'
)
fig1.show()

fig2 = px.imshow(
    cov_ewma,
    labels={'x':'Factor i','y':'Factor j','color':'Covariance'},
    title='EWMA Covariance (h=63d)'
)
fig2.show()

fig3 = px.imshow(
    cov_shrunk,
    labels={'x':'Factor i','y':'Factor j','color':'Covariance'},
    title='Shrunk EWMA Covariance (λ=0.8)'
)
fig3.show()

In [12]:
pfa.portfolio_weights

BBG000N9MNX3    0.074286
BBG000B9XRY4    0.098472
BBG000BVPV84    0.161695
BBG009S39JX6    0.218856
BBG000BBJQV0    0.194392
BBG000BPH459    0.208248
BBG000MM2P62    0.044051
Name: weight, dtype: float64

In [ ]:
def implied_returns_from_portfolio(cov_matrix: pd.DataFrame,
                                   portfolio_weights: pd.Series,
                                   risk_aversion: float = 2.5) -> pd.Series:
    """
    Inverse-optimisation: recover the expected-return vector μ that would make
    `portfolio_weights` the mean–variance optimal portfolio for a given
    risk-aversion γ.

    Mean–variance FOC
    -----------------
        μ  =  γ · Σ · w_opt

    Parameters
    ----------
    cov_matrix : pd.DataFrame
        Σ  (assets × assets or factors × factors).
    portfolio_weights : pd.Series
        w_opt  (must align with Σ's index).
    risk_aversion : float, default 2.5
        γ  (higher ⇒ more risk-averse).

    Returns
    -------
    pd.Series
        Implied expected returns μ (same index as Σ).
    """
    # align
    w = portfolio_weights.reindex(cov_matrix.index).fillna(0.0).values
    mu = risk_aversion * (cov_matrix.values @ w)
    return pd.Series(mu, index=cov_matrix.index, name="weights")

In [ ]:
from factorinvesting.src.portfolio_models import FixedCovariance,EmpiricalPrior,FixedMu

implied_factor_return_df=implied_returns_from_portfolio(portfolio_weights=pfa.portfolio_exposure_df.loc[last_date],
                               cov_matrix=cov_shrunk,
                               )





In [ ]:
from skfolio.utils.equations import equations_to_matrix

# ---------- 1. Convert text views to P & q ----------------------
def to_pick_matrix(groups: np.ndarray,
                   equations: list[str],
                   *,
                   sum_to_one: bool = True,
                   raise_if_group_missing: bool = False):
    """
    Wrapper around `equations_to_matrix` that returns the BL
    equality-view objects P (k×n) and q (k,).
    Any inequality views are returned separately so you can
    pass them as hard constraints to the optimiser.
    """
    A_eq, b_eq, A_le, b_le = equations_to_matrix(
        groups=groups,
        equations=equations,
        sum_to_one=sum_to_one,
        raise_if_group_missing=raise_if_group_missing,
    )

    # Equality views are P r == q
    P, q = A_eq, b_eq

    # A_le r <= b_le can be held for the optimiser
    return P, q, A_le, b_le
# ---------- 2. Meucci-style Black-Litterman core ----------------
def black_litterman(pi: np.ndarray,
                    Sigma: np.ndarray,
                    P: np.ndarray,
                    q: np.ndarray,
                    c: float | np.ndarray = 0.10):
    """
    Posterior mean & covariance, Meucci parametrisation.
    """
    if P.size == 0:          # no equality views
        return pi.copy(), Sigma.copy()

     # --- Ω according to Meucci: Ω_i = (1-c_i)/c_i · diag(PΣPᵀ)_i
    diag_prior = np.diag(P @ Sigma @ P.T)         # k-vector
    # handle scalar vs vector c
    ratio = (1.0 - c) / c                         # → 0 when c = 1
    Omega = np.diag(ratio * diag_prior)

    M = P @ Sigma @ P.T + Omega
    K = np.linalg.inv(M)

    mu_bl = pi + Sigma @ P.T @ K @ (q - P @ pi)
    Sigma_bl = Sigma - Sigma @ P.T @ K @ P @ Sigma

    return mu_bl, Sigma_bl
# ---------- 3. One-liner convenience ----------------------------
def bl_from_text_views(pi, Sigma, equations, c=0.10, **kw):
    """
    Full pipeline: text ➜ matrices ➜ BL posterior.
    Returns:
        mu_bl, Sigma_bl, A_le, b_le
    The last two can be added as constraints in most optimisers.
    """
    groups=np.asarray([Sigma.columns.tolist()])
    P, q, A_le, b_le = to_pick_matrix(groups, equations, **kw)

    mu_bl, Sigma_bl = black_litterman(pi.values, Sigma.values, P, q, c)

    mu_bl=pd.Series(mu_bl,index=implied_factor_return_df.index,name="weights")
    Sigma_bl=pd.DataFrame(index=cov_shrunk.index,columns=cov_shrunk.columns,data=Sigma_bl)

    return mu_bl, Sigma_bl, A_le, b_le

In [ ]:

equations = [
    "beta == 20",      # inequality, goes to the optimiser
]

# 2. Posterior inputs for the optimiser



bl_returns, bl_cov, A_le, b_le = bl_from_text_views(
    implied_factor_return_df, cov_shrunk,  equations, c=0.0000010, sum_to_one=True
)

In [20]:

bl_returns-implied_factor_return_df

lncap             2.046867e-06
lncap2           -4.674738e-06
beta              1.999998e-05
nl_beta          -1.432097e-05
mom_12_1          1.114186e-06
mom_1m           -6.515877e-07
resid_vol         2.279348e-06
liquidity        -2.642523e-08
book_to_price     6.207512e-07
leverage         -1.532950e-07
div_yield         5.002779e-07
earnings_yield    1.040438e-06
profitability    -4.574611e-07
Name: weights, dtype: float64

In [21]:

def factor_to_asset_df(mu_f: pd.Series,
                       Sigma_f: pd.DataFrame,
                       B: pd.DataFrame,
                       alpha: pd.Series | None = None,
                       spec_var: pd.Series | None = None
                      ) -> tuple[pd.Series, pd.DataFrame]:
    """
    Expand a *factor* model (in pandas objects) to *asset* mean & covariance.

    Parameters
    ----------
    mu_f      : Series      (k,)      expected factor returns
    Sigma_f   : DataFrame   (k,k)     factor covariance
    B         : DataFrame   (n,k)     asset × factor exposure matrix
    alpha     : Series      (n,)      asset alphas; default 0
    spec_var  : Series      (n,)      specific variances σ²_spec; default 0

    Returns
    -------
    mu_a      : Series      (n,)      asset expected returns
    Sigma_a   : DataFrame   (n,n)     asset covariance matrix
    """

    # 1. Align factor names
    factor_names = mu_f.index
    Sigma_f = Sigma_f.reindex(index=factor_names, columns=factor_names)
    B       = B.reindex(columns=factor_names).fillna(0.0)

    # 2. Align asset names
    asset_names = B.index
    if alpha is None:
        alpha = pd.Series(0.0, index=asset_names)
    else:
        alpha = alpha.reindex(asset_names).fillna(0.0)

    if spec_var is None:
        spec_var = pd.Series(0.0, index=asset_names)
    else:
        spec_var = spec_var.reindex(asset_names).fillna(0.0)

    # 3. Compute
    mu_a    = B.dot(mu_f) + alpha
    factor_cov_asset = B.dot(Sigma_f).dot(B.T)
    D       = pd.DataFrame(np.diag(spec_var), index=asset_names, columns=asset_names)
    Sigma_a = factor_cov_asset + D

    return mu_a, Sigma_a

asset_returns,asset_covariance=factor_to_asset_df(mu_f=bl_returns, Sigma_f=bl_cov,
                                               B=pfa.exposures_df.loc[last_date],
                                               )

In [22]:
asset_returns

unique_identifier
BBG000BS5CM9    0.000354
BBG01Q08MMR1    0.000181
BBG01NK9ZXT4    0.000298
BBG01G9JKWV5    0.000361
BBG0113JGQF0    0.000235
                  ...   
BBG000BKFZM4    0.000244
BBG000BKL348    0.000233
BBG000BKR1D6    0.000185
BBG000BKTFN2    0.000227
BBG000BKVJK4    0.000332
Length: 476, dtype: float64

In [23]:
asset_covariance

unique_identifier,BBG000BS5CM9,BBG01Q08MMR1,BBG01NK9ZXT4,BBG01G9JKWV5,BBG0113JGQF0,BBG00YRJXKB1,BBG00MRHG523,BBG00KXRCDP0,BBG00HVVB499,BBG00FQH6BS9,...,BBG000BJSBJ0,BBG000BK2F42,BBG000BK67C7,BBG000BK6MB5,BBG000BKCFC2,BBG000BKFZM4,BBG000BKL348,BBG000BKR1D6,BBG000BKTFN2,BBG000BKVJK4
unique_identifier,,,,,,,,,,,,,,,,,,,,,
BBG000BS5CM9,-0.000265,-0.000173,-0.000263,-0.000344,-0.000257,-0.000377,-0.000307,-0.000262,-0.000228,-0.000269,...,-0.000063,-0.000186,-0.000146,-3.012328e-05,-0.000251,-0.000184,-0.000300,-0.000178,-0.000249,-0.000364
BBG01Q08MMR1,-0.000173,-0.000097,-0.000155,-0.000195,-0.000138,-0.000212,-0.000163,-0.000139,-0.000130,-0.000159,...,-0.000042,-0.000109,-0.000089,-6.334163e-07,-0.000147,-0.000107,-0.000171,-0.000094,-0.000145,-0.000210
BBG01NK9ZXT4,-0.000263,-0.000155,-0.000224,-0.000268,-0.000216,-0.000283,-0.000252,-0.000209,-0.000204,-0.000246,...,-0.000050,-0.000156,-0.000128,-5.992906e-07,-0.000214,-0.000150,-0.000261,-0.000141,-0.000222,-0.000313
BBG01G9JKWV5,-0.000344,-0.000195,-0.000268,-0.000237,-0.000238,-0.000258,-0.000292,-0.000226,-0.000256,-0.000324,...,-0.000058,-0.000180,-0.000170,4.112084e-05,-0.000263,-0.000151,-0.000328,-0.000158,-0.000292,-0.000359
BBG0113JGQF0,-0.000257,-0.000138,-0.000216,-0.000238,-0.000125,-0.000234,-0.000217,-0.000164,-0.000191,-0.000248,...,-0.000067,-0.000160,-0.000147,5.351021e-05,-0.000223,-0.000135,-0.000241,-0.000126,-0.000206,-0.000272
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BBG000BKFZM4,-0.000184,-0.000107,-0.000150,-0.000151,-0.000135,-0.000144,-0.000176,-0.000141,-0.000149,-0.000182,...,-0.000038,-0.000104,-0.000102,2.500008e-05,-0.000172,-0.000078,-0.000194,-0.000091,-0.000168,-0.000210
BBG000BKL348,-0.000300,-0.000171,-0.000261,-0.000328,-0.000241,-0.000388,-0.000266,-0.000214,-0.000198,-0.000260,...,-0.000051,-0.000178,-0.000131,-2.928234e-06,-0.000206,-0.000194,-0.000264,-0.000168,-0.000226,-0.000347
BBG000BKR1D6,-0.000178,-0.000094,-0.000141,-0.000158,-0.000126,-0.000153,-0.000149,-0.000125,-0.000135,-0.000159,...,-0.000040,-0.000101,-0.000088,8.671101e-06,-0.000138,-0.000091,-0.000168,-0.000073,-0.000148,-0.000196


In [24]:
implied_asset_return=implied_factor_return_df=implied_returns_from_portfolio(portfolio_weights=pfa.portfolio_weights,
                               cov_matrix=asset_covariance,
                               )
(implied_asset_return-asset_returns)

unique_identifier
BBG000BS5CM9   -0.000016
BBG01Q08MMR1   -0.000008
BBG01NK9ZXT4   -0.000011
BBG01G9JKWV5   -0.000009
BBG0113JGQF0   -0.000008
                  ...   
BBG000BKFZM4   -0.000012
BBG000BKL348   -0.000007
BBG000BKR1D6   -0.000005
BBG000BKTFN2   -0.000009
BBG000BKVJK4   -0.000012
Length: 476, dtype: float64

In [25]:
implied_asset_return

unique_identifier
BBG000BS5CM9    0.000338
BBG01Q08MMR1    0.000173
BBG01NK9ZXT4    0.000287
BBG01G9JKWV5    0.000352
BBG0113JGQF0    0.000227
                  ...   
BBG000BKFZM4    0.000232
BBG000BKL348    0.000226
BBG000BKR1D6    0.000180
BBG000BKTFN2    0.000218
BBG000BKVJK4    0.000320
Name: weights, Length: 476, dtype: float64

In [26]:
asset_returns

unique_identifier
BBG000BS5CM9    0.000354
BBG01Q08MMR1    0.000181
BBG01NK9ZXT4    0.000298
BBG01G9JKWV5    0.000361
BBG0113JGQF0    0.000235
                  ...   
BBG000BKFZM4    0.000244
BBG000BKL348    0.000233
BBG000BKR1D6    0.000185
BBG000BKTFN2    0.000227
BBG000BKVJK4    0.000332
Length: 476, dtype: float64